In [2]:
#!/usr/bin/env python
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String
from sqlalchemy import sql, select, join, desc

# Create a Engine object which is our handle into the database.
engine = create_engine('sqlite:///.../world.sqlite')

# Connect to the database
conn = engine.connect()

# Read the metadata from the existing database.
#  Since the database already exists and has tables defined, we can create Python objects based on these automatically.
DBInfo=MetaData(engine)

In [4]:
# Create a country object based on the metadata
country=Table('country', DBInfo, autoload=True)


#SELECT country code, name, and life expectancy FROM country 
#WHERE life expectancy  > 80 ORDER BY Life Expectancy DESC LIMIT 10;

query=select([country.c.Code, country.c.Name, country.c.LifeExpectancy])\
             .where(country.c.LifeExpectancy > 80)\
             .order_by(desc(country.c.LifeExpectancy))\
             .limit(10)
result = conn.execute(query)
for row in result:
    print(row)
print(result)

('AND', 'Andorra', Decimal('83.5000000000'))
('MAC', 'Macao', Decimal('81.6000000000'))
('SMR', 'San Marino', Decimal('81.1000000000'))
('JPN', 'Japan', Decimal('80.7000000000'))
('SGP', 'Singapore', Decimal('80.1000000000'))


In [5]:
#SELECT country code, name, and life expectancy FROM country 
#WHERE life expectancy  > 80 ORDER BY Life Expectancy DESC LIMIT 10;
#JOIN countrylanguage ON Code=Country Name 
#WHERE Language='Portuguese' ORDER BY Percentage DESC;

country=Table('country', DBInfo, autoload=True)
countrylanguage=Table('countrylanguage', DBInfo, autoload=True)

query=(select([country.c.Name, country.c.LifeExpectancy, countrylanguage.c.Language, countrylanguage.c.Percentage])\
      .select_from(country.join(countrylanguage)).where(country.c.LifeExpectancy > 80).where(countrylanguage.c.Percentage > 40)\
      .order_by(desc(country.c.LifeExpectancy)))
result = conn.execute(query)
for row in result:
    print(row)

('Andorra', Decimal('83.5000000000'), 'Spanish', Decimal('44.6000000000'))
('Macao', Decimal('81.6000000000'), 'Canton Chinese', Decimal('85.6000000000'))
('San Marino', Decimal('81.1000000000'), 'Italian', Decimal('100.0000000000'))
('Japan', Decimal('80.7000000000'), 'Japanese', Decimal('99.1000000000'))
('Singapore', Decimal('80.1000000000'), 'Chinese', Decimal('77.1000000000'))


In [6]:
import pandas as pd

data = {'Country': ['Andora', 'Macao','San Marino', 'Japan','Singapore'],
        'Life_Expectancy': [83.50, 81.60, 81.10, 80.70, 80.10],
        'Language': ['Spanish', 'Canton Chinese', 'Italian', 'Japanese', 'Chinese']
        }

df = pd.DataFrame(data)

print(df)

      Country  Life_Expectancy        Language
0      Andora             83.5         Spanish
1       Macao             81.6  Canton Chinese
2  San Marino             81.1         Italian
3       Japan             80.7        Japanese
4   Singapore             80.1         Chinese


In [7]:
#Updating a database: first location who the Head of state is in Canada in this table
#Charles III is the new head of state 

query=select([country.c.Code, country.c.Name, country.c.HeadOfState]).where(country.c.Name == 'Canada')
result = conn.execute(query)
for row in result:
    print(row)

('CAN', 'Canada', 'Elisabeth II')
